Here I'll shortly describe the final solution, and if there is interest
I could go more extensively in details on the different optimization
steps I took in the course of the challenge.

The main observations however are as follows:
1. Solving the smaller equivalent problem with C1=0, i.e. (0, C2-C1, C_max-sum(C1)).
2. Using a single QFT/IQFT transform and performing all additions in QFT space.
3. Reducing/replacing the initial QFT to/with H.
4. Using the most significant `carry` bit in the data register (2**c - C_max -1 + cost) directly as a flag qubit and c=data_bits-1.
5. Using approximate QFT with approximation_degree 1 or 2.
6. Using transpile(['cx','u3'], optimization_level=3)

**N.B.** you can also check the notes and comments in my [submission](challenge_4_kpe.py).

In [2]:
# lets do the imports

from typing import Union
import math
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Gate

Let's start by describing the general structure of the solution circuit
(proposed in [arXiv:1908.02210](https://arxiv.org/abs/1908.02210).

We represent the $N$ choices needed to be made with $N$ index qubits (for all
the knapsack problem instances in this IQC problem $N=\text{index_qubits}=11$).

We then repetitively apply two operators - $U(\gamma)$ und $U(\beta)$,
 with different values for $\gamma$ and $\beta$,
 in each iteration, such that initially $\gamma$ is small and $\beta$ large,
 and with their relative strength gradually changing towards $\gamma$ being
 large and $\beta$ being small. (intuitively this corresponds to gradually
 decreasing the system temperature, until the system anneals to its lowest
 energy state - but I'm not quantum physicist, so this intuition might be
 incorrect)

By all means we only have:
 * a quantum circuit with
   * 11 index qubits
   * 5 data qubits  - for calculating the cost of a solution
 * a for loop to repetitively apply the $U(\gamma)$ and $U(\beta)$ operators:
   * $U(\gamma)$ application consist of two parts:
     * phase return part
     * cost penalty part - for this we need the data qubits
       * cost calculation using QRAM to encode the cost of each choice
       * constraint testing to flag all infeasible solutions (where the cost constraint is not satisfied)
       * penalty dephasing of the infeasible solutions
       * reinitialization of the data qubits using the inverse operation, so they can be used in the next iteration
   * $U(\beta)$ application (mixing operator)

So the overall structure of the circuit would be like this:

```python
# problem definition
L1, L2 = [],[]         # list of value returns for associated with each choice
C1, C2 = [],[]         # list of cost associated with each choice

C_max: int             # the maximal cost of a feasible solution

# circuit definition
index_qubits = len(L1) # number of choices to be made
data_qubits:int        # number of data qubits required to calculate a solution cost

qr_index = QuantumRegister(index_qubits, "index")
qr_data  = QuantumRegister(data_qubits, "data")
cr_index = ClassicalRegister(index_qubits, "c_index")
qc = QuantumCircuit(qr_index, qr_data, cr_index)

qc.h(qr_index)  # put the index/solution qubits in a superposition

p, alpha = 5, 1
for i in range(p):
    beta, gamma = 1 - (i + 1) / p, (i + 1) / p  # calculate beta and gamma

    ### return part ###
    qc.append(phase_return(index_qubits, gamma, L1, L2), qr_index)

    ### penalty part ###

    ### step 1: cost calculation ###
    qc.append(cost_calculation(index_qubits, data_qubits, C1, C2), qr_index[:] + qr_data[:])

    ### step 2: Constraint testing ###
    qc.append(constraint_testing(data_qubits, C_max), qr_data[:] + qr_f[:])

    ### step 3: penalty dephasing ###
    qc.append(penalty_dephasing(data_qubits, alpha, gamma), qr_data[:] + qr_f[:])

    ### step 4: reinitialization ###
    qc.append(reinitialization(index_qubits, data_qubits, C1, C2, C_max), qr_index[:] + qr_data[:] + qr_f[:])

    ### mixing operator ###
    qc.append(mixing_operator(index_qubits, beta), qr_index)

### measure the index ###
qc.measure(qr_index, cr_index[::-1])
```

The phase return part is quite straightforward:

In [3]:
### Phase Operator ###
# return part
def phase_return(index_qubits: int, gamma: float, L1: list, L2: list, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_index = QuantumRegister(index_qubits, "index")
    qc = QuantumCircuit(qr_index)

    for ndx in range(index_qubits):
        qc.p(- gamma * (L2[ndx] - L1[ndx]), qr_index[ndx])

    return qc.to_gate(label=" phase return ") if to_gate else qc

Implementing the QFT adder can be done by simply following
the illustration figures in [arXiv:quant-ph/0008033](https://arxiv.org/pdf/quant-ph/0008033.pdf).

Getting the QFT Adder right is probably the most complex part of
the solution - so we better unit test it well. (I can provide my unit tests later).

(We could have also used the [Qiskit QFTAdder](https://qiskit.org/documentation/stubs/qiskit.circuit.library.DraperQFTAdder.html)
or the [Qiskit QFT](https://qiskit.org/documentation/stubs/qiskit.circuit.library.QFT.html) directly,
but as I have found out, it might be somewhat confusing to get the qubit order right, so it was easier
for me to just follow the illustrations in
[arXiv:quant-ph/0008033](https://qiskit.org/documentation/stubs/qiskit.circuit.library.QFT.html))

In [ ]:
QFT_AD = 1   # default approximation degree of the QFT

#
# QFT implementation (with qiskit bit ordering).
#
def qft(data_qubits: int, approximation_degree: int = QFT_AD, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr = QuantumRegister(data_qubits)
    qc = QuantumCircuit(qr)

    for target in reversed(range(data_qubits)):
        qc.h(target)
        for k, control in enumerate(reversed(range(target)), start=2):
            if k > (data_qubits - approximation_degree):
                continue
            qc.cp(2 * np.pi / (2 ** k), control, target)

    return qc.to_gate(label=" qft ") if to_gate else qc

#
# Adds a constant to tha data register in QFT space.
# Should be used with the above QFT implementation.
#
def subroutine_add_const(data_qubits: int, const: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_data = QuantumRegister(data_qubits)
    qc = QuantumCircuit(qr_data)

    # prepares the const bits in a list
    cbits = list(map(int, reversed(f"{np.abs(const):02b}".zfill(data_qubits))))

    for target in reversed(range(data_qubits)):
        tsum = 0  # cumulative phase for target
        for k, control in enumerate(reversed(range(target + 1)), start=1):
            cbit = cbits[control]
            if cbit:
                tsum += 1 / 2 ** k
        if tsum > 0:
            qc.p(np.sign(const) * 2 * np.pi * tsum, qr_data[target])

    return qc.to_gate(label=" [+" + str(const) + "] ") if to_gate else qc

#
# the const parameter in the above subroutine_add_const() could be negative,
# but I have never really used this, so feel free to remove the np.abs and np.sign above.
#

Using the `qft()` and `subroutine_add_const()` form above, we could
define `const_adder()` and `cost_calculation()` as below, but in our
final optimized solution we would notice, that we can do all additions
in QFT space, thus having only one QFT transform in each iteration.

In [ ]:
#
# This is how a **single** QFT addition of a const to the
# data register would look like, however we'll not use this method,
# but instead do a single QFT, do all phase rotations there, before
# doing the inverse IQFT transformation.
# (This is actually the main idea in optimizing the 4c solution.)
#
def const_adder(data_qubits: int, const: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_data = QuantumRegister(data_qubits)
    qc = QuantumCircuit(qr_data)

    qc.append(qft(data_qubits), qr_data[:])
    qc.append(subroutine_add_const(data_qubits, const, to_gate=False).to_gate(), qr_data[:])
    qc.append(qft(data_qubits).inverse(), qr_data[:])

    return qc.to_gate(label=" [+" + str(const) + "] ") if to_gate else qc

#
# This is how the QRAM cost calculation function would look like, however
# we will not use it directly, but do all additions in QFT space, wrapped
# in a single QFT.
#
def cost_calculation(index_qubits: int, data_qubits: int, list1: list, list2: list, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_index = QuantumRegister(index_qubits, "index")
    qr_data = QuantumRegister(data_qubits, "data")
    qc = QuantumCircuit(qr_index, qr_data)

    # note the -1 bellow - the cost would fit in (data_qubits-1) bits,
    # and we'll use the most significant bit to flag solutions with an infeasible cost.
    for i, (val1, val2) in enumerate(zip(list1, list2)):
        qc.append(const_adder(data_qubits - 1, val2).control(1), [qr_index[i]] + qr_data[:-1])
        qc.x(qr_index[i])
        qc.append(const_adder(data_qubits - 1, val1).control(1), [qr_index[i]] + qr_data[:-1])
        qc.x(qr_index[i])

    return qc.to_gate(label=" Cost Calculation ") if to_gate else qc

#
# This is how `constraint_testing` could be implemented, when
# using the above `cost_calculation()`.
# (We'll be using a single QFT however - see the next `cost_calculation()` implementation below)
#
def constraint_testing(data_qubits: int, C_max: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_data = QuantumRegister(data_qubits, "data")
    qr_f = QuantumRegister(1, "flag")
    qc = QuantumCircuit(qr_data, qr_f)

    c = data_qubits - 1
    w = 2**c - (C_max+1)
    qc.append(const_adder(data_qubits, w), qr_data[:])  # offset by w = 2**c -(C_max+1) to flag infeasible costs in MSB

    # set the flag qubit when the MSB is set
    # qc.cx(qr_data[-1], qr_f) # we'll be using the MSB directly without the explicit qr_f qubit

    return qc.to_gate(label=" Constraint Testing ") if to_gate else qc

And now we present the final optimized version of `cost_calculation()`,
where only a single QFT is used, to do all cost additions in QFT space. We'll
also perform the additon of the offset factor `w` here, so that we can do all
additions with a single QFT.

The overall structure of the `cost_calculation()` would be like:
```python
c = data_qubits - 1
w = 2 ** c - (C_max + 1)

# QFT
qc.append(qft(data_qubits), qr_data[:])  # qft

# QRAM
for i, (val1, val2) in enumerate(zip(list1, list2)):
    assert val1 == 0
    qc.append(subroutine_add_const(data_qubits, val2).control(1), [qr_index[i]] + qr_data[:])

# offset by w to flag infeasible costs directly with the data_qubits MSB
qc.append(subroutine_add_const(data_qubits, w), qr_data[:])

# inverse QFT
qc.append(qft(data_qubits).inverse(), qr_data[:])

```

We can now note, that the initial state of all data qubits is `|+> = H|0>`,
and by observing the QFT figure in [arXiv:quant-ph/0008033](https://qiskit.org/documentation/stubs/qiskit.circuit.library.QFT.html)
we note, that the QFT acts on
each data qubits by applying a H-Gate first, followed by conditional rotations.
 However this second application of
the H-Gate sets the data qubit back in the |0> state, so that the following
conditional rotations have no effect. Therefore, this initial QFT (acting on data
qubits in the `|+>` state) can be replaced by a single H-Gate.

In [ ]:
#
# This is the actual cost_calculation() used in the final solution:
#
# Here we'll complete the QRAM cost addition to the data register
# in QFT space and also add the w = (2^c - (C_max+1)) offset term,
# so that the most significant bit of the data register is set to 1 whenever
# the cost in the data register exceeds C_max (i.e. cost > C_max).
#
# N.B. even the paper uses a (cost >= C_max) condition to set a penalty
# of alpha*(cost - C_max), the penalty would be zero for cost == C_max,
# therefore we use the strict inequality.
#
def cost_calculation(index_qubits: int, data_qubits: int, list1: list, list2: list, to_gate: bool = True) -> Union[Gate, QuantumCircuit]:
    qr_index = QuantumRegister(index_qubits, "index")
    qr_data = QuantumRegister(data_qubits, "data")
    qc = QuantumCircuit(qr_index, qr_data)

    ## lets mark only cost > C_max

    c = data_qubits - 1      # use the MSB in data directly as a flag qubit
    w = 2 ** c - (C_max + 1)

    ###########
    ## QFT start - do all additions in QFT space
    # qc.append(qft(data_qubits), qr_data[:])
    qc.h(qr_data[:])  # initially all data qubits are |0> so qft is just Hadamard

    # QRAM
    for i, (val1, val2) in enumerate(zip(list1, list2)):
        assert val1 == 0
        qc.append(subroutine_add_const(data_qubits, val2).control(1), [qr_index[i]] + qr_data[:])

    qc.append(subroutine_add_const(data_qubits, w), qr_data[:])  # offset by w = 2**c -(C_max+1) to flag infeasible costs in MSB

    qc.append(qft(data_qubits).inverse(), qr_data[:])
    ## QFT end
    ####################

    return qc.to_gate(label=" Cost Constraint Testing ") if to_gate else qc

#
# After returning from QFT space, we can use the MSB of the data register
# to set the flag qubit (for infeasible solutions), but instead
# we'll be using the MSB directly.
#
def constraint_testing(data_qubits: int, C_max: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_data = QuantumRegister(data_qubits, "data")
    qr_f = QuantumRegister(1, "flag")
    qc = QuantumCircuit(qr_data, qr_f)

    # qc.cx(qr_data[-1], qr_f) # we'll be using the MSB directly without the explicit qr_f qubit

    return qc.to_gate(label=" Constraint Testing ") if to_gate else qc

The `penalty_dephasing()` is straight forward following Figure 13 in [arXiv:1908.02210](https://arxiv.org/abs/1908.02210).

In [ ]:
# penalty part
def penalty_dephasing(data_qubits: int, alpha: float, gamma: float, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_data = QuantumRegister(data_qubits, "data")
    qr_f = QuantumRegister(1, "flag")
    qc = QuantumCircuit(qr_data, qr_f)

    c = data_qubits - 1

    #
    # we use the qr_data[-1] as a flag directly
    #
    qr_f = qr_data[-1]

    for k in range(c):
        qc.cp(alpha * gamma * (2**k), qr_f, qr_data[k])
    qc.p(-alpha * gamma * (2**c), qr_f)

    return qc.to_gate(label=" Penalty Dephasing ") if to_gate else qc

We then use `reinitialization()` to inverse the calculation on the
data and flag (well we don't use that ove) qubits, by applying the inverse transforms done so far.

In [ ]:
# penalty part
def reinitialization(index_qubits: int, data_qubits: int, C1: list, C2: list, C_max: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_index = QuantumRegister(index_qubits, "index")
    qr_data = QuantumRegister(data_qubits, "data")
    qr_f = QuantumRegister(1, "flag")
    qc = QuantumCircuit(qr_index, qr_data, qr_f)

    # constrain_testing is empty see above
    qc.append(constraint_testing(data_qubits, C_max).inverse(), qr_data[:] + qr_f[:])
    qc.append(cost_calculation(index_qubits, data_qubits, C1, C2, to_gate=True).inverse(), qr_index[:] + qr_data[:])

    return qc.to_gate(label=" Reinitialization ") if to_gate else qc

The `mixing_operator()` is also quit straight forward, following the paper:

In [ ]:
### Mixing Operator ###
def mixing_operator(index_qubits: int, beta: float, to_gate=True) -> Union[Gate, QuantumCircuit]:
    qr_index = QuantumRegister(index_qubits, "index")
    qc = QuantumCircuit(qr_index)

    for ndx in range(index_qubits):
        qc.rx(2 * beta, ndx)

    return qc.to_gate(label=" Mixing Operator ") if to_gate else qc

Now before combining all we have so far, lets prepare a python decorator, we'll be
using to decorate our `solver_function()` so it gets optimized by Qiskit:

In [ ]:
OPT_LEVEL = 3

def transpile(qc: QuantumCircuit) -> QuantumCircuit:
    # this was added, but can be disabled with setting OPT_LEVEL to -1 above.
    if OPT_LEVEL != -1:
        qc = qiskit.transpile(qc, basis_gates=['cx', 'u3'], seed_transpiler=42, optimization_level=OPT_LEVEL)
    return qc


def transpile_optimize(fn):
    def wrapper(*args, **kwargs):
        return transpile(fn(*args, **kwargs))
    return wrapper

This way we end up with the following `solver_function()` function, also used for the final submission:

In [ ]:
from typing import Union
import math

import numpy as np

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Gate

# True for using the explicit flag qubit, False for using the MSB in data register
# when USE_FLAG_QUBIT==True - we need only (data_qubits-1) qubits
USE_SCORING    = "new"  # new, old

USE_5_QUBITS   = True   # True to use 1 qubit less than the template suggestion
QFT_AD         = 1      # QFT approximation degree (number of terms to omit)
OPT_LEVEL      = 3      # transpiler optimization level (0-3) or -1 to disable

if USE_SCORING == 'new':
    QFT_AD       = 2
    USE_5_QUBITS = True
elif USE_SCORING == 'old':
    QFT_AD       = 1
    USE_5_QUBITS = False
else:
    raise "Unexpected value for USE_SCORING (expecting one of {new, old})"




def transpile(qc: QuantumCircuit) -> QuantumCircuit:
    # this was added, but can be disabled with setting OPT_LEVEL to -1 above.
    if OPT_LEVEL != -1:
        qc = qiskit.transpile(qc, basis_gates=['cx', 'u3'], seed_transpiler=42, optimization_level=OPT_LEVEL)
    return qc


def transpile_optimize(fn):
    def wrapper(*args, **kwargs):
        return transpile(fn(*args, **kwargs))
    return wrapper


@transpile_optimize
def solver_function(L1: list, L2: list, C1: list, C2: list, C_max: int) -> QuantumCircuit:
    """
    Solves the 4c knapsack problem (assuming C1<C2, and L1<L2).
    """

    # print name and score
    author = 'Kamen Petroff (kpe)'
    score = 'old:260_084 new:3_482_625'
    print(f'{author}: {score}')

    # first let's convert it to the equivalent problem with cost (0, C2-C1)
    C1, C2 = np.array(C1), np.array(C2)
    C1, C2, C_max = C1 - C1, C2 - C1, C_max - C1.sum()

    # the number of qubits representing answers
    index_qubits = len(L1)

    # the maximum possible total cost
    max_c = sum([max(l0, l1) for l0, l1 in zip(C1, C2)])

    # the number of qubits representing data values can be defined using the maximum possible total cost as follows:
    data_qubits = math.ceil(math.log(max_c, 2)) + 1 if not max_c & (max_c - 1) == 0 else math.ceil(math.log(max_c, 2)) + 2

    if USE_5_QUBITS:
        data_qubits -= 1

    ### Phase Operator ###
    # return part
    def phase_return(index_qubits: int, gamma: float, L1: list, L2: list, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qc = QuantumCircuit(qr_index)

        for ndx in range(index_qubits):
            qc.p(- gamma * (L2[ndx] - L1[ndx]), qr_index[ndx])

        return qc.to_gate(label=" phase return ") if to_gate else qc

    #
    # QFT implementation (with qiskit bit ordering).
    #
    def qft(data_qubits: int, approximation_degree: int = QFT_AD, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr = QuantumRegister(data_qubits)
        qc = QuantumCircuit(qr)

        for target in reversed(range(data_qubits)):
            qc.h(target)
            for k, control in enumerate(reversed(range(target)), start=2):
                if k > (data_qubits - approximation_degree):
                    continue
                qc.cp(2 * np.pi / (2 ** k), control, target)

        return qc.to_gate(label=" qft ") if to_gate else qc

    #
    # Adds a constant to tha data register in QFT space.
    # Should be used with the above QFT implementation.
    #
    def subroutine_add_const(data_qubits: int, const: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_data = QuantumRegister(data_qubits)
        qc = QuantumCircuit(qr_data)

        # prepares the const bits in a list
        cbits = list(map(int, reversed(f"{np.abs(const):02b}".zfill(data_qubits))))

        for target in reversed(range(data_qubits)):
            tsum = 0  # cumulative phase for target
            for k, control in enumerate(reversed(range(target + 1)), start=1):
                cbit = cbits[control]
                if cbit:
                    tsum += 1 / 2 ** k
            if tsum > 0:
                qc.p(np.sign(const) * 2 * np.pi * tsum, qr_data[target])

        return qc.to_gate(label=" [+" + str(const) + "] ") if to_gate else qc

    #
    # This is how a **single** QFT addition of a const to the
    # data register would look like, however we'll not use this method,
    # but instead do a single QFT, do all phase rotations there, before
    # doing the inverse IQFT transformation.
    # (This is actually the main idea in optimizing the 4c solution.)
    #
    def const_adder(data_qubits: int, const: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_data = QuantumRegister(data_qubits)
        qc = QuantumCircuit(qr_data)

        qc.append(qft(data_qubits), qr_data[:])
        qc.append(subroutine_add_const(data_qubits, const, to_gate=False).to_gate(), qr_data[:])
        qc.append(qft(data_qubits).inverse(), qr_data[:])

        return qc.to_gate(label=" [+" + str(const) + "] ") if to_gate else qc

    #
    # This is how the QRAM cost calculation function would look like, however
    # we will not use it directly, but do all additions in QFT space, wrapped
    # in a single QFT.
    #
    def _cost_calculation(index_qubits: int, data_qubits: int, list1: list, list2: list, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qc = QuantumCircuit(qr_index, qr_data)

        # note the -1 bellow - the cost would fit in (data_qubits-1) bits,
        # and we'll use the most significant bit to flag solutions with an infeasible cost.
        for i, (val1, val2) in enumerate(zip(list1, list2)):
            qc.append(const_adder(data_qubits - 1, val2).control(1), [qr_index[i]] + qr_data[:-1])
            qc.x(qr_index[i])
            qc.append(const_adder(data_qubits - 1, val1).control(1), [qr_index[i]] + qr_data[:-1])
            qc.x(qr_index[i])

        return qc.to_gate(label=" Cost Calculation ") if to_gate else qc

    #
    # This is the actual cost_calculation() used:
    #
    # Here we'll complete the QRAM cost addition to the data register
    # in QFT space and also add a (2^c - (C_max+1)) term, so that the
    # most significant bit of the data register is set to 1 whenever
    # the cost in the data register exceeds C_max (i.e. cost > C_max).
    #
    # N.B. even the paper uses a (cost >= C_max) condition to set a penalty
    # of alpha*(cost - C_max), the penalty would be zero for cost == C_max,
    # therefore we use a strict inequality.
    #
    def cost_calculation(index_qubits: int, data_qubits: int, list1: list, list2: list, to_gate: bool = True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qc = QuantumCircuit(qr_index, qr_data)

        ## lets mark only cost > C_max

        c = data_qubits - 1      # use the MSB in data directly as a flag qubit
        w = 2 ** c - (C_max + 1)

        ###########
        ## QFT start - do all additions in QFT space
        # qc.append(qft(data_qubits), qr_data[:])
        qc.h(qr_data[:])  # initially all data qubits are |0> so qft is just Hadamard

        # QRAM
        for i, (val1, val2) in enumerate(zip(list1, list2)):
            assert val1 == 0
            qc.append(subroutine_add_const(data_qubits, val2).control(1), [qr_index[i]] + qr_data[:])

    	qc.append(subroutine_add_const(data_qubits, w), qr_data[:])  # offset by w = 2**c -(C_max+1) to flag infeasible costs in MSB

        qc.append(qft(data_qubits).inverse(), qr_data[:])
        ## QFT end
        ####################

        return qc.to_gate(label=" Cost Constraint Testing ") if to_gate else qc

    #
    # After returning from QFT space, we can use the MSB of the data register
    # to set the flag qubit (for infeasible solutions).
    #
    def constraint_testing(data_qubits: int, C_max: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_data, qr_f)


        # qc.cx(qr_data[-1], qr_f) # we'll be using the MSB directly without the explicit qr_f qubit

        return qc.to_gate(label=" Constraint Testing ") if to_gate else qc

    # penalty part
    def penalty_dephasing(data_qubits: int, alpha: float, gamma: float, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_data, qr_f)

        c = data_qubits - 1

        #
        # we use the qr_data[-1] as a flag directly
        #
        qr_f = qr_data[-1]

        for k in range(c):
            qc.cp(alpha * gamma * (2**k), qr_f, qr_data[k])
        qc.p(-alpha * gamma * (2**c), qr_f)

        return qc.to_gate(label=" Penalty Dephasing ") if to_gate else qc

    # penalty part
    def reinitialization(index_qubits: int, data_qubits: int, C1: list, C2: list, C_max: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_index, qr_data, qr_f)

        # constrain_testing is empty see above
        qc.append(constraint_testing(data_qubits, C_max).inverse(), qr_data[:] + qr_f[:])
        qc.append(cost_calculation(index_qubits, data_qubits, C1, C2, to_gate=True).inverse(), qr_index[:] + qr_data[:])

        return qc.to_gate(label=" Reinitialization ") if to_gate else qc

    ### Mixing Operator ###
    def mixing_operator(index_qubits: int, beta: float, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qc = QuantumCircuit(qr_index)

        for ndx in range(index_qubits):
            qc.rx(2 * beta, ndx)

        return qc.to_gate(label=" Mixing Operator ") if to_gate else qc

    #
    # everything bellow this line was not touched, with exception
    # of the transpiling/optimization in the last line which can be
    # controlled using the OPT_LEVEL=-1 define above.
    #
    ##############################

    qr_index = QuantumRegister(index_qubits, "index")  # index register
    qr_data = QuantumRegister(data_qubits, "data")  # data register
    qr_f = QuantumRegister(1, "flag")  # flag register
    cr_index = ClassicalRegister(index_qubits, "c_index")  # classical register storing the measurement result of index register
    qc = QuantumCircuit(qr_index, qr_data, qr_f, cr_index)

    ### initialize the index register with uniform superposition state ###
    qc.h(qr_index)

    ### DO NOT CHANGE THE CODE BELOW
    p = 5
    alpha = 1
    for i in range(p):
        ### set fixed parameters for each round ###
        beta = 1 - (i + 1) / p
        gamma = (i + 1) / p

        ### return part ###
        qc.append(phase_return(index_qubits, gamma, L1, L2), qr_index)

        ### step 1: cost calculation ###
        qc.append(cost_calculation(index_qubits, data_qubits, C1, C2), qr_index[:] + qr_data[:])

        ### step 2: Constraint testing ###
        qc.append(constraint_testing(data_qubits, C_max), qr_data[:] + qr_f[:])

        ### step 3: penalty dephasing ###
        qc.append(penalty_dephasing(data_qubits, alpha, gamma), qr_data[:] + qr_f[:])

        ### step 4: reinitialization ###
        qc.append(reinitialization(index_qubits, data_qubits, C1, C2, C_max), qr_index[:] + qr_data[:] + qr_f[:])

        ### mixing operator ###
        qc.append(mixing_operator(index_qubits, beta), qr_index)

    ### measure the index ###
    ### since the default measurement outcome is shown in big endian, it is necessary to reverse the classical bits in order to unify the endian ###
    qc.measure(qr_index, cr_index[::-1])

    return qc